PPCA lavora con più variabili osservabili anzichè una sola

In [1]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy import stats
import scipy
import pandas as pd
import warnings

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
scaler = StandardScaler()

# extract data social anxiety

In [3]:
csv_ = '/Users/marcoghezzi/PycharmProjects/pythonProject/osfstorage-archive/behavior/LookAtMe_045.csv'
csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_045.csv'
global_data = pd.read_csv(csv_, sep='\t')
y = np.array(list([int(d>2) for d in global_data['rating']]))
e_labels = y[:,np.newaxis]  # rating > 2

In [5]:
N_e = e_labels.shape[0]
D_e = e_labels.shape[1]
print(N_e,D_e)

160 1


## test with new data

In [6]:
from deepemogp import feature_extractor
from deepemogp.signal import physio as physio
from deepemogp import datasets as datasets
from deepemogp.signal import behavior as behavior
from deepemogp.signal.physio import ecg
# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np

show = False
# definition of the feature extractors to be used later
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(1,0))

# definition of the physiological signals to be extracted
eda_ = physio.EDA(f2)
hr_ = physio.HR(f2)
pupil_ = behavior.PUPIL(f3)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr_,pupil_,eda_}, subjects={'45'})

for s in d.signals:
    # preprocess ...
    if s.name =='EDA':
        s.preprocess(show=show,new_fps=500)
        s.feature_ext.extract_feat(s,show=show)
    else:
        if s.name == 'HR':
            list_hr_test = s.raw[0]['data']
            s.preprocess(show=show, useneurokit=True)
            s.feature_ext.extract_feat(s,show=show)

        else:
            s.feature_ext.extract_feat_without_preprocess(s, show=show)

    #add feature extraction for eda before preprocessing

    # ... and extract features from each signal type


for sig in d.signals:
    if sig.name=='EDA':
        eda_data = sig.features
    if sig.name=='HR':
        hr_data = sig.features
    if sig.name=='PUPIL':
        pupil_data = sig.features

>> Loading EDA for subject 45 and all sessions from dataset FEAR
>> Loading HR for subject 45 and all sessions from dataset FEAR
>> Loading PUPIL for subject 45 and all sessions from dataset FEAR
>> Processing EDA ...
>> Extracting wavelet features from EDA signal, adopting (2, 1) window ...
>> Processing HR ... using neurokit


/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/neurokit2/epochs/epochs_create.py:205: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for colname, column in epochs[i].select_dtypes(include=["object"]).iteritems():
/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/neurokit2/epochs/epochs_create.py:205: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for colname, column in epochs[i].select_dtypes(include=["object"]).iteritems():
/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/neurokit2/epochs/epochs_create.py:205: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for colname, column in epochs[i].select_dtypes(include=["object"]).iteritems():
/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/n

>> Extracting wavelet features from HR signal, adopting (2, 1) window ...
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...


In [12]:
TRIAL = 160

In [13]:
hr =np.array(hr_data)
hr = hr.reshape((TRIAL, int(hr.shape[0]/TRIAL*hr.shape[1])))

array([[1507.82432239, 1507.82432239, 1507.82432239, ..., 1566.0845009 ,
        1520.02818112, 1518.7678512 ],
       [1561.94779093, 1561.94779093, 1561.94779093, ..., 1513.92693106,
        1511.8541424 , 1511.8541424 ],
       [1552.65898888, 1552.65898888, 1552.65898888, ..., 1478.91710429,
        1478.91614366, 1478.91614366],
       ...,
       [1513.53959853, 1513.53959853, 1513.53959853, ..., 1475.70110856,
        1475.70110856, 1475.70110856],
       [1562.66691975, 1562.66691975, 1562.66691975, ..., 1694.83054188,
        1731.68433009, 1731.6894679 ],
       [1539.28006789, 1539.28006789, 1539.28006789, ..., 1445.25223889,
        1441.56087692, 1441.25889603]])

In [14]:
pupil = np.array(pupil_data)
pupil = pupil.reshape((TRIAL, int(pupil.shape[0]/TRIAL*pupil.shape[1])))

array([[2.45239393, 2.66638125, 2.59187033, 2.4506036 , 2.80837978],
       [2.59338696, 2.55991296, 2.53492629, 2.48206472, 2.62299583],
       [2.51776189, 2.52536855, 2.71710262, 2.85383682, 2.64254111],
       [2.59916026, 2.78052864, 2.66078013, 2.80587036, 2.90862783],
       [2.66953859, 2.56049477, 2.37752592, 2.4753718 , 2.82322928],
       [2.69363716, 2.60489613, 2.52116537, 2.56927633, 2.71404879],
       [2.53859531, 2.63078144, 3.00423614, 2.67140556, 2.65304165],
       [2.45419989, 2.29169615, 2.40401754, 2.40156297, 2.46497346],
       [2.7961956 , 2.60416088, 2.46037891, 2.54203925, 2.48787623],
       [2.46366409, 2.67486512, 2.65886801, 2.5452179 , 2.69123385],
       [2.35282239, 2.51053252, 2.47818746, 2.61634768, 2.34638445],
       [2.69495718, 2.91349163, 3.06015049, 2.77073565, 2.72773236],
       [2.53074146, 2.42715984, 2.60548436, 2.70087583, 2.3135078 ],
       [2.55345281, 2.62065232, 2.58789636, 2.644122  , 2.72449498],
       [2.45065309, 2.5222609 , 2.

In [15]:
eda = np.array(eda_data)
eda = eda.reshape((TRIAL,int(eda.shape[0]/TRIAL*eda.shape[1])))

array([[ 2.10891983e-01,  2.12998437e-01,  1.82950635e-01, ...,
         1.31000424e+01,  1.19876484e+01,  1.09912966e+01],
       [ 4.79881800e-01,  4.84782808e-01,  4.19306285e-01, ...,
         1.70782579e+01,  1.52385652e+01,  1.37090198e+01],
       [-8.62379325e-01, -8.71626109e-01, -7.66350705e-01, ...,
         1.88873483e+01,  1.80621614e+01,  1.71648583e+01],
       ...,
       [-8.87428418e-01, -8.97424247e-01, -8.04640781e-01, ...,
         1.75345145e+01,  2.41071828e+01,  2.83073815e+01],
       [ 1.27413860e+00,  1.28461551e+00,  1.04262039e+00, ...,
         3.05011220e+01,  3.90704780e+01,  4.29526697e+01],
       [-1.25129379e-02, -1.24530688e-02, -5.70494938e-03, ...,
         2.95990803e+01,  4.42385068e+01,  5.23625619e+01]])

## end test

prendo un'implementazione di sppca e la applico al nostro caso

different types of observable data

1) gaze
2) fisio
    2.1) heart rate variability
    2.2) eda phasic value
3) social anxiety
4) aspettativa del dolore

In [16]:

'''social_anx = data['social anxiety'][:,np.newaxis]
pain_exp = data['pain expectation'][:,np.newaxis]'''


"social_anx = data['social anxiety'][:,np.newaxis]\npain_exp = data['pain expectation'][:,np.newaxis]"

In [18]:
N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]
K = 3

print(N_pupil,D_pupil)
print(N_hr,D_hr)
print(N_eda,D_eda)
print(N_e, D_e)

160 5
160 60
160 60
160 1


In [19]:
with pm.Model(coords={"time": hr}) as sPPCA:
    #dati osservabili
    hr_data = pm.MutableData("hr_data", hr.T)
    pupil_data = pm.MutableData("pupil_data", pupil.T)
    eda_data = pm.MutableData("eda_data", eda.T)

    e_data = pm.ConstantData("e_data", e_labels.T)

    #matrici pesi
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]), shape=[D_pupil, K])

    Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #weight matrix for pain expectation.
    #check mu,sigma,shape
    We = pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e,K]), shape=[D_e, K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([N_hr,K]), sigma=at.ones([N_hr,K]), shape=[N_hr,K])

    # dati dell'hrv interpretati come una gaussiana
    sigma_alpha_hr = pm.Exponential("sigma_alpha", 50.0)
    x_hr = pm.GaussianRandomWalk('x_hr', sigma=sigma_alpha_hr, init_dist=pm.Normal.dist(mu=0,sigma=1,dims = 'time'))

    '''
    x_hr = pm.Normal('x_hr', mu=Whr.dot(c.T), sigma=at.ones([D_hr,N_hr]) ,shape=[D_hr, N_hr], observed=hr_data)
    '''

    # dati della dilatazione pupille interpretati come una gaussiana
    x_pupil = pm.Normal('x_pupil', mu=Wpupil.dot(c.T), sigma=at.ones([D_pupil, N_pupil]), shape=[D_pupil, N_pupil], observed=pupil_data)
    #eda
    x_eda = pm.Normal('x_eda',mu=Weda.dot(c.T),sigma= at.ones([D_eda,N_pupil]), shape=[D_eda,N_eda], observed=eda_data)

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    x_e = pm.Bernoulli('x_e' , p=pm.math.sigmoid(We.dot(c.T)) , shape =[D_e, N_e], observed=e_data)

    #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

NotImplementedError: The use of a `.dist(dims=...)` API is not supported.

In [ ]:
with pm.Model(coords={"time": hr}) as sPPCA:
    #dati osservabili
    hr_data = pm.MutableData("hr_data", hr.T)
    e_data = pm.ConstantData("e_data", e_labels.T)

    #matrici pesi
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])

    We = pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e,K]), shape=[D_e, K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([N_hr,K]), sigma=at.ones([N_hr,K]), shape=[N_hr,K])

    # dati dell'hrv interpretati come una gaussiana
    sigma_alpha_hr = pm.Exponential("sigma_alpha", 50.0)
    x_hr = pm.GaussianRandomWalk('x_hr', sigma=sigma_alpha_hr, init_dist=pm.Normal.dist(mu=Whr.dot(c.T),sigma=sigma=at.ones([D_hr,N_hr])),dims = 'time')

    x_e = pm.Bernoulli('x_e' , p=pm.math.sigmoid(We.dot(c.T)) , shape =[D_e, N_e], observed=e_data)

In [ ]:
gv = pm.model_to_graphviz(sPPCA)
gv.view()

In [ ]:
sPPCA.free_RVs

In [ ]:
with sPPCA:
    approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    trace = approx.sample(500)

'''with sPPCA:
    trace = pm.sample(1000,init='advi+adapt_diag',chains=1,target_accept=0.95)'''

In [ ]:
#az.plot_posterior(mean_field.sample(1000), color="LightSeaGreen")
az.plot_trace(trace)
plt.show()

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"pupil_data": pupil,"hr_data":hr,"eda_data":eda})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)

In [ ]:
e_pred = posterior_predictive.posterior_predictive["x_e"]

In [ ]:
e_pred_mode = np.squeeze(stats.mode(e_pred[0], keepdims=False)[0])[:,np.newaxis]

In [ ]:
e_pred_mode

In [ ]:
e_labels

In [ ]:
train_accuracy_exp = accuracy_score(e_labels, e_pred_mode)
print('\n\tTrain Accuracy Pain Expectation: ' + str(train_accuracy_exp))
print(' ')

--------------------------------------------------------------------------------------------

In [ ]:
hr_pred_mean = (stats.median_abs_deviation(hr_pred[0]))[0]
hr_pred_mean

In [ ]:
hr[0]

In [ ]:
#hr_pred_mode= pd.Series([0.006,0.01,0.02,0.003])

hr and hr_pred_mode devono avere same shape

In [ ]:
train_accuracy_hr = (np.subtract(hr[0], hr_pred_mean))
print('\n\t avg distance hr: \n' + str(train_accuracy_hr))
#print(' ')

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"hr_data": hr})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
    gaze_pred = posterior_predictive.posterior_predictive["x_gaze"]

In [ ]:
gaze_pred

In [ ]:
gaze_pred_mean = np.mean(gaze_pred, axis=0).T.reshape(-1)
gaze_true = pupil.reshape(-1)
reconstructed_r2 = metrics.r2_score(gaze_true, gaze_pred_mean)

print('\n\tReconstructed X R2 score: ' + str(reconstructed_r2))

In [ ]:
import deepemogp.feature_extractor as feature_extractor

In [ ]:
f = feature_extractor.FE('wavelet', window=(8,6), params={'w_mother':'db3','w_maxlev':1})

In [ ]:
x = [3, 7, 1, 1, -2, 5, 4, 6]

In [ ]:
f.apply(x)